### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [8]:
import os
# REFs:
#https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/tools/
#https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent_parallel_function_calling/
#https://docs.llamaindex.ai/en/stable/understanding/putting_it_all_together/agents/
from llama_index.llms.oci_genai import OCIGenAI
from llama_index.core.agent import ReActAgent #Framework to work with agents - they use a lot of intermediate prompts
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import ChatMessage
import pandas as pd

In [9]:
compartment_id = os.environ["NB_SESSION_COMPARTMENT_OCID"]
command_r = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"
command_r_plus = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya7ozidbukxwtun4ocm4ngco2jukoaht5mygpgr6gq2lgq"

In [10]:
def checa_rodada(inputs: int) -> str:
    """Checa os resultados a partir do número da rodada representada por um número inteiro"""
  
    df = pd.read_csv('Tabela_Futebol_Agents.csv')
    
    rodada = df[df['Rodada'] == int(inputs)]
    
    time_A = rodada['Time_A'].astype(str)
    time_B = rodada['Time_B'].astype(str)
    
    gol_A = rodada['Gol_TimeA'].astype(str)
    gol_B = rodada['Gol_TimeB'].astype(str)
    
    result = f"{time_A.iloc[0]} {gol_A.iloc[0]} x {gol_B.iloc[0]} {time_B.iloc[0]}"
    
    return result

def checa_times(inputs: str) -> str:
    """Checa os resultados a partir de 1 dos times que jogaram. Os nomes sempre começam com letra maiúscula.
    Alguns exemplos de nomes de times: Vasco, São Paulo, Corinthians, Sport.
    """
    
    df = pd.read_csv('Tabela_Futebol_Agents.csv')
    
    time = inputs
    
    lista_time_A = df['Time_A'].unique()
    lista_time_B = df['Time_B'].unique()

    if time in lista_time_A:
        time_A = time
        time_B = df.loc[df['Time_A'] == time, 'Time_B'].iloc[0]
        gol_A = df.loc[df['Time_A'] == time, 'Gol_TimeA'].iloc[0]
        gol_B = df.loc[df['Time_A'] == time, 'Gol_TimeB'].iloc[0]

        result = f"{time_A} {gol_A} x {gol_B} {time_B}"

    elif time in lista_time_B:
        time_B = time
        time_A = df.loc[df['Time_B'] == time, 'Time_A'].iloc[0]
        gol_A = df.loc[df['Time_B'] == time, 'Gol_TimeA'].iloc[0]
        gol_B = df.loc[df['Time_B'] == time, 'Gol_TimeB'].iloc[0]

        result = f"{time_A} {gol_A} x {gol_B} {time_B}"

    return result

# Define the tools
tools = [checa_rodada, checa_times]

functions = []
for tool in tools:
    functions.append(FunctionTool.from_defaults(tool)) #  FunctionTool é um contêiner usado para integrar essas funções como ferramentas para o agente

# Initialize LLM with OCI Generative AI
llm = OCIGenAI(
    model=command_r_plus,#"command_r_plus", #https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    provider="cohere",
    context_size="128000",
    temperature=0.1,
    max_tokens=2500
)

# Initialize ReAct agent with the add tool
agent = ReActAgent.from_tools(functions, llm=llm, verbose=True)

In [11]:
ReActAgent.from_tools

<bound method ReActAgent.from_tools of <class 'llama_index.core.agent.react.base.ReActAgent'>>

In [20]:
# Use the agent to perform an addition operation
prompt =  """
Qual foi o resultado do jogo do Vasco?
"""

result = agent.chat(prompt) #chat(str(messages))
print(result.response)

Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: checa_times
Action Input: {'inputs': 'Vasco'}
Observation: Vasco 0 x 2 Flamengo
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: O Vasco perdeu de 0 a 2 para o Flamengo.
O Vasco perdeu de 0 a 2 para o Flamengo.


In [19]:
result.response

'O resultado da segunda rodada foi Vasco 0 x 2 Flamengo.'